In [1]:
import numpy as np
import pandas as pd
import pickle
import json
import plotly
import plotly.express as px
import plotly.graph_objs as gokay
from plotly.subplots import make_subplots
import os
import tensorflow as tf
import urllib.request
from PIL import Image
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from glob import glob
from skimage.transform import resize
import random
from tqdm import tqdm
from skimage.io import imread, imshow, concatenate_images
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt
import plotly.graph_objs as go
plt.style.use("ggplot")
import cv2


CATEGORIES = ['Begnin case', 'Malignant case', 'Normal case']

app = Flask(__name__)
app.secret_key = "secret key"

MODEL_PATH ='output/cnn_model.h5'
cnn = load_model(MODEL_PATH)
model = load_model('output/unet_model.h5')

model1= pickle.load(open('output/output.pkl', 'rb'))
model2 = pickle.load(open('output/result.pkl', 'rb'))



OUTPUT_FOLDER = os.path.join('static', 'output')
app.config['UPLOAD_FOLDER'] = OUTPUT_FOLDER

@app.route('/')
def index():
    return render_template('index.html') 

@app.route('/symptoms')
def symptoms():
    return render_template('symptoms.html')

@app.route('/mainmedicalinsurance')
def mainmedicalinsurance():
    return render_template('mainmedicalinsurance.html') 

@app.route('/lungcancer')
def lungcancer():
    return render_template('lungcancerdetection.html')

@app.route('/chart1//')
def chart1():
    data = pd.read_csv('output/dataset1.csv')
    trace0 = go.Histogram(x=data['GENDER'],name="Gender")
    trace1 = go.Histogram(x=data['AGE'],name="Age")
    trace2 = go.Histogram(x=data['SMOKING'],name="Smoking")
    trace3 = go.Histogram(x=data['YELLOW_FINGERS'],name="Yellow Fingers")
    trace5 = go.Histogram(x=data['ANXIETY'],name="Anxiety")
    trace6 = go.Histogram(x=data['PEER_PRESSURE'],name="Peer Pressure")
    trace7 = go.Histogram(x=data['CHRONIC_DISEASE'],name="Chronic Disease")
    trace8 = go.Histogram(x=data['FATIGUE'],name="Fatigue")
    trace9 = go.Histogram(x=data['ALLERGY'],name="Allergy")
    trace10 = go.Histogram(x=data['WHEEZING'],name="Wheezing")
    trace11= go.Histogram(x=data['ALCOHOL_CONSUMING'],name="Alcohol Consuming")
    trace12= go.Histogram(x=data['COUGHING'],name="Coughing")
    trace13= go.Histogram(x=data['SHORTNESS_OF_BREATH'],name="Shortness Of Breath")
    trace14= go.Histogram(x=data['SWALLOWING_DIFFICULTY'],name="Swallowing Difficulty")
    trace15= go.Histogram(x=data['CHEST_PAIN'],name="Chest Pain")
    trace16 = go.Histogram(x=data['LUNG_CANCER'],name="Lung Cancer")
    data1 = pd.read_csv('output/dataset2.csv')
    corrmat = data1.corr()
    trace17 = go.Heatmap( z = corrmat.values, x = list(corrmat.columns),y = list(corrmat.index),colorscale = 'Viridis',showscale=False)
    models = pd.read_csv('output/models.csv')
    trace18 = go.Bar(x=models['Model'], y= models['Score'],name="Model Comparison")

    fig = plotly.tools.make_subplots(
        rows=9,
        cols=3,
        specs=[[{}, {}, {}],[{},{},{}],[{}, {'colspan': 2, 'rowspan': 3}, None], [{} , None, None],[{} , None, None],[{}, {}, {}],[{},{},{}],[{'colspan': 3, 'rowspan': 1},None, None],[{'colspan': 3, 'rowspan': 1},None, None]],
        subplot_titles=('Gender','Allergy', 'Smoking',"Yellow Fingers", "Anxiety","Peer Pressure","Chronic Disease","Age","Fatigue","Lung Cancer","Wheezing","Alcohol Consuming","Coughing","Shortness Of Breath","Swallowing Difficulty","Chest Pain","Correlation Matrix"," Model Comparison")
        
    )
    fig.update_layout(width=1100,height=3000,title="Data Distribution, X-axis indicates 'Features' and Y-axis indicates 'Count'")

    fig.append_trace(trace0, 1, 1)
    fig.append_trace(trace9, 1, 2)
    fig.append_trace(trace2, 1, 3)
    fig.append_trace(trace3, 2, 1)
    fig.append_trace(trace5, 2, 2)
    fig.append_trace(trace6, 2, 3)
    fig.append_trace(trace7, 3, 1)
    fig.append_trace(trace1, 3, 2)
    fig.append_trace(trace8, 4, 1)
    fig.append_trace(trace16, 5, 1)
    fig.append_trace(trace10, 6, 1)
    fig.append_trace(trace11, 6, 2)
    fig.append_trace(trace12, 6, 3)
    fig.append_trace(trace13, 7, 1)
    fig.append_trace(trace14, 7, 2)
    fig.append_trace(trace15, 7, 3)
    fig.append_trace(trace17, 8,1)
    fig.append_trace(trace18, 9,1)
    fig.update_layout(bargap=0.2)
    
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    header="Lung Cancer Analysis"
    description="The Random Forest Classifer is the most accurate model for predicting Lung Cancer"
    return render_template('notdash.html', graphJSON=graphJSON,header=header,description=description)

@app.route('/chart2')
def chart2():
    data = pd.read_csv('output/insurance.csv')
    trace0 = go.Histogram(x=data['age'],name="AGE")
    trace1 = go.Histogram(x=data['sex'],name="SEX")
    trace2 = go.Histogram(x=data['bmi'],name="BMI")
    trace3 = go.Histogram(x=data['children'],name="CHILDREN")
    trace4 = go.Histogram(x=data['smoker'],name="SMOKER")
    trace5 = go.Histogram(x=data['region'],name="REGION")
    trace6 = go.Histogram(x=data['charges'],name="CHARGES")
    trace11 = go.Scatter(x=data['age'], y=data['bmi'], name="AGExBMI",mode='markers')
    trace12 = go.Scatter(x=data['age'], y=data['children'], name="AGExCHILDREN", mode='markers')
    trace13= go.Scatter(x=data['age'], y=data['charges'], name="AGExCHARGES", mode='markers')
    trace14= go.Scatter(x=data['bmi'], y=data['charges'],  name="BMIxCHARGES",mode='markers')
    trace15= go.Scatter(x=data['children'], y=data['charges'],  name="CHILDRENxCHARGES",mode='markers')
    trace16= go.Scatter(x=data['smoker'], y=data['charges'],  name="SMOKERxCHARGES",mode='markers')
    corrmat = data.corr()
    trace21=  go.Heatmap( z = corrmat.values, x = list(corrmat.columns),y = list(corrmat.index),colorscale = 'Viridis',showscale=False)
    
    models = pd.read_csv('output/models2.csv')
    trace22 = go.Bar(x=models['Model'], y= models['Score'],name="Model Comparison",marker = {'color' : 'teal'})

    fig = plotly.tools.make_subplots(
        rows=11,
        cols=3,
        specs=[[{'rowspan':2},{},{'rowspan':2}],[None,{},None],[{},{},{}],[{'colspan':3,'rowspan':3},None,None],[None,None,None],[None,None,None],[{},{},{}],[{},{},{}],[{'colspan':3,'rowspan':3},None,None],[None,None,None],[None,None,None]],
        subplot_titles=('CHARGES','SEX','BMI','CHILDREN','SMOKER','REGION','AGE','CORRELATION MATRIX','AGExBMI','AGExCHILDREN','AGExCHARGES','BMIxCHARGES','CHILDRENxCHARGES','SMOKERxCHARGES','MODELS COMPARISON')
        
    )
    
    fig.update_layout(width=1300,height=1300)
    fig.append_trace(trace0, 3, 3)
    fig.append_trace(trace1, 1, 2)
    fig.append_trace(trace2, 1, 3)
    fig.append_trace(trace3, 3, 1)
    fig.append_trace(trace4, 2, 2)
    fig.append_trace(trace5, 3, 2)
    fig.append_trace(trace6, 1, 1)
    fig.append_trace(trace11, 7, 1)
    fig.append_trace(trace12, 7, 2)
    fig.append_trace(trace13, 7, 3) 
    fig.append_trace(trace14, 8, 1)
    fig.append_trace(trace15, 8, 2)  
    fig.append_trace(trace16, 8, 3)
    fig.append_trace(trace21, 4, 1)
    fig.append_trace(trace22, 9, 1)

    fig.update_layout(bargap=0.2,title="Data Distribution, X-axis indicates 'Features' and Y-axis indicates 'Count'")
    header="Medical Insurance for Lung Cancer"
    description="The Random Forest Regression is the best for obtaining medical insurance"
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return render_template('notdash.html', graphJSON=graphJSON,header=header,description=description)

@app.route('/predicts',methods=['POST','GET'])
def predicts():
    int_features = [int(x) for x in request.form.values()]
    if(len(int_features)>0):
        final_features = [np.array(int_features)]
        prediction = model1.predict(final_features)
        output = prediction[0]
        print(output)
        if output == 1:
            res_val = "Lung cancer"
            return render_template('stage.html', prediction_text='Patient may have {}'.format(res_val))
        else:
            res_val = "Lung cancer"
            return render_template('healthy.html', prediction_text='Patient may not have {}'.format(res_val))
    else:
            return render_template('stage.html')

@app.route('/predict1',methods=['POST','GET'])
def predict1():
  int_features1 = [int(x) for x in request.form.values()]
  final_features1 = [np.array(int_features1)]
  prediction = model2.predict(final_features1)
  output = prediction[0]
  return render_template('insuranceoutput.html', prediction_text='Medical insurance cost is {}'.format(round(output,2)))

@app.route('/medicalinsurance/')
def medicalinsurance1():
    return render_template('insurance.html')


def prepare(filename):
    IMG_SIZE=256
    img =cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
    img_array = img / 255.0
    new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
    img_tensor = new_array.reshape(-1,IMG_SIZE,IMG_SIZE,1)                             
    return img_tensor

@app.route("/stage", methods=['GET', 'POST'])
def stage():
    return render_template('stage.html')   

@app.route('/upload_image/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':      
        file = request.files['file']
        filename = file.filename
        #Classification
        img_path= os.path.join(r'static/uploads/',filename)
        file.save(img_path)
        print(filename)
        prediction = cnn.predict(prepare(img_path))
        #flash(prediction)
        predictions = np.argmax(prediction,-1)
        #flash(predictions[0])
        pred = CATEGORIES[int(predictions[0])]
        if(pred=='Malignant case'):
            flash(pred + " suggests that the tumour MAYBE harmful")
            flash("Please click on the 'Nodule Detection' button ")
          
        else:
            flash(pred + " suggests that the tumour MAY NOT BE harmful")
            flash("Although the type is not harmful, it is recommended to get nodule detetcion done")
            flash("Please click on the 'Nodule Detection' button  for the same")
        return render_template('typedetection.html', filename=filename,prediction_text='Type Detected: {}'.format(pred))

 
@app.route('/display/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename='uploads/' + filename), code=301)

@app.route("/unet", methods=['GET', 'POST'])
def unet():
    return render_template('unet.html')


def plot_sample(X, y, preds, binary_preds, ix=None):
    """Function to plot the results"""
    if ix is None:
        ix = random.randint(0, len(X))

    has_mask = y[ix].max() > 0

    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0], cmap='seismic')
    if has_mask:
        ax[0].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[0].set_title('Lung CT Scan ')

    ax[1].imshow(y[ix].squeeze())
    ax[1].set_title('Ground Truth')

    ax[2].imshow(preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[2].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[2].set_title('Predicted Nodule')
    
    ax[3].imshow(binary_preds[ix].squeeze(), vmin=0, vmax=1)
    if has_mask:
        ax[3].contour(y[ix].squeeze(), colors='k', levels=[0.5])
    ax[3].set_title('Predicted Nodule Binary')
    results_dir = os.path.join('static/output/')
    sample_file_name = "final_result.png"
    print(results_dir + sample_file_name)
    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)
    fig.savefig(results_dir+sample_file_name)


def unet_prediction(image):
    test_images  = glob('E:/Major Project/prepared_data/test/images/*.png')
    test_masks = glob('E:/Major Project/prepared_data/test/masks/*.png')

    img = []
    for filename in test_images:
        img.append(filename)
    img.sort()
    mask = []
    for filename in test_masks:
        mask.append(filename)
    mask.sort()
    im_width = 512
    im_height = 512
    border = 5
    ids = next(os.walk("E:/Major Project/prepared_data/test/images"))[2] # list of names all images in the given path
    print("No. of images = ", len(ids))
    X = np.zeros((1, im_height, im_width, 1), dtype=np.float32)
    y = np.zeros((1, im_height, im_width, 1), dtype=np.float32)
    images_path = "E:/Major Project/prepared_data/test/images/" 
    masks_path = "E:/Major Project/prepared_data/test/masks/"
    img = load_img(images_path+image, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (512, 512, 1), mode = 'constant', preserve_range = True)
    # Load masks
    mask = img_to_array(load_img(masks_path+image, grayscale=True))
    mask = resize(mask, (512, 512, 1), mode = 'constant', preserve_range = True)
        # Save images
    X[0] = x_img/255.0
    y[0] = mask/255.0
    preds_val = model.predict(X, verbose=1)
    preds_val_t = (preds_val > 0.5).astype(np.uint8)
    # Check if valid data looks all right
    plot_sample(X, y, preds_val, preds_val_t)


@app.route("/predict", methods = ['GET','POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        filename = file.filename
        file_path = os.path.join(r'E:/Major Project/static/predict', filename)                       #slashes should be handeled properly
        file.save(file_path)
        print(filename)
        print(file_path)
        product = unet_prediction(filename)
        print(product) 
        full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'final_result.png')
        print(full_filename)
        return render_template("predict.html", user_image = full_filename)

if __name__ == '__main__': 
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.1 when using version 0.24.2. This might lead to breaking co

1


127.0.0.1 - - [23/May/2022 11:43:55] "POST /predicts HTTP/1.1" 200 -


0


127.0.0.1 - - [23/May/2022 11:43:58] "GET /stage HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 11:44:04] "GET /mainmedicalinsurance HTTP/1.1" 200 -
c:\Users\User\anaconda3\lib\site-packages\plotly\tools.py:460: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

127.0.0.1 - - [23/May/2022 11:44:07] "GET /chart2 HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 11:44:10] "GET /medicalinsurance/ HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 11:44:41] "POST /predict1 HTTP/1.1" 200 -
